# SynD Model Creation and Trajectory Generation
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jdrusso/SynD/HEAD?labpath=examples%2Fsynd_demo.ipynb)

In [1]:
from synd.models.discrete.markov import MarkovGenerator
from synd.core import load_model
import pickle
from scipy import sparse
import numpy as np

### Define backmapping

Here, the backmapping is just defined as a dictionary.

Each discrete state has a unique progress coordinate value, computed from the atomistic structures.
Note that there's no reason this has to be a 1D progress coordinate -- this can map discrete states to 3D atomic structures, too.
Or, instead of a 1:1 mapping, you could (at the cost of a performance hit) do more complex things, such as slightly modifying the structure before mapping it, or mapping states to out-of-sample structures with something like Deep Generative MSMs.

In [2]:
pcoord_map_path = 'data/trp-cage/pcoord_map.pkl'

with open(pcoord_map_path, 'rb') as infile:
    pcoord_map = pickle.load(infile)
    
backmapper = pcoord_map.get

In [3]:
pcoord_map[0]

array([0.19300674])

### Define transition matrix

This is a 10,500-state transition matrix, built from a long atomistic Trp-cage MD trajectory.

In [4]:
transition_matrix_path = 'data/trp-cage/sparse_tmatrix.npz'

transition_matrix = sparse.load_npz(transition_matrix_path)

### Create and save SynD model

In [5]:
synd_model = MarkovGenerator(
    transition_matrix=transition_matrix,
    backmapper=backmapper,
    seed=None
)

In [6]:
synd_model.save('trp-cage.synd')

## Generate data

Let's generate a 200us trajectory...

In [7]:
%%time

trajectory = synd_model.generate_trajectory(
    initial_states=np.array([1871]),
    n_steps=200000
)
trajectory

CPU times: user 1.86 s, sys: 0 ns, total: 1.86 s
Wall time: 1.86 s


array([[1871,  583,  728, ..., 9356, 8555, 8555]])

## Backmap to progress coordinate

In [8]:
synd_model.backmap(trajectory)

array([[4.68604740e-05, 1.94142014e-01, 1.29532188e-01, ...,
        5.40355384e-01, 8.78116250e-01, 8.78116250e-01]])

## Load saved SynD model, and generate data

In [9]:
loaded_model = load_model('trp-cage.synd')

In [10]:
loaded_model.generate_trajectory(
    initial_states = np.array([1871, 2003]),
    n_steps = 5
)

array([[1871,  583,  728,  847,  809],
       [2003,  991,  324,  102,  870]])